# Deep Learning MTH 5320 
## Homework 3
## 12/6/2023
## By Josias Moukpe

## Object Detection Model Assignment

### Dataset Prep and Preprocessing (20 points)
- Collect or create a dataset of images containing the objects you want to detect annotated with bounding boxes.
- Implement data preprocessing techniques, including:
  - Resizing images
  - Normalizing pixels
- Hint: You can find pre-annotated datasets for download on [Roboflow](https://github.com/roboflow/supervision).

### Model Architecture (30 points)
- Train a YOLOv8 model on your selected dataset using the public repository available at [Ultralytics](https://github.com/ultralytics/ultralytics).
- Monitor the training progress and use appropriate techniques to prevent overfitting.
- Conduct at least 3 tuning experiments.

In [29]:
# Import libraries
import torch
from ultralytics import YOLO
import datetime

In [30]:
# Check if GPU is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using {device} device')
if device.type == 'cuda':
    print(torch.cuda.get_device_name(0))

Using cuda device
NVIDIA GeForce RTX 3060


In [31]:
# configs
DEVICE = 0 if device.type == 'cuda' else 'cpu'
SEED = 42

In [32]:
# OCR dataset downloaded from Roboflow in Yolo v8 format
local_ocr_ds_path = 'D:/College/Fall2023/DeepLearning/yolo_ocr'
local_ocr_ds_yaml_path = 'D:/College/Fall2023/DeepLearning/yolo_ocr/data.yaml'
PROJECT_NAME = 'yolov8n_ocr'

In [33]:
# display the yaml file
with open(local_ocr_ds_yaml_path) as f:
    print(f.read())

train: ../train/images
val: ../valid/images
test: ../test/images

nc: 36
names: ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z']

roboflow:
  workspace: w251ocr
  project: ocr-oy9a7
  version: 1
  license: CC BY 4.0
  url: https://universe.roboflow.com/w251ocr/ocr-oy9a7/dataset/1


##### Experiment 1
- Testing baseline default hyperparameters
- To prevent overfitting, I use early stopping, weight decay, and dropout
- Image resizing (to IMGSZ) and normalization (to [0, 1]) is handle by ultralytics library


In [36]:
# loading pre-trained yolov8 model from ultralytics
model_exp1 = YOLO('yolov8n.pt')
# display the model architecture
model_exp1

YOLO(
  (model): DetectionModel(
    (model): Sequential(
      (0): Conv(
        (conv): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(16, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (1): Conv(
        (conv): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (2): C2f(
        (cv1): Conv(
          (conv): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
          (act): SiLU(inplace=True)
        )
        (cv2): Conv(
          (conv): Conv2d(48, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_s

In [ ]:
# format timestamp as string
timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
# hyperparameters
IMGSZ = 640                       # Image resolution for training
EPOCHS = 10                       # Number of training epochs
BATCH = 8                         # Batch size for training
PATIENCE = 50                     # Early stopping patience in epochs
EXP_NAME = f'baseline_yolov8n_ocr_{timestamp}'  # Experiment name with timestamp
OPTIMIZER = 'AdamW'               # Optimization algorithm
VERBOSE = True                    # Display detailed training info
FRACTION_TO_TRAIN = 1.0           # Fraction of training set to use
InitLR = 1e-2                     # Initial learning rate
FinalLR = 1e-2                    # Final learning rate
MOMENTUM = 0.937                  # Momentum for optimizer
WEIGHT_DECAY = 5e-4               # Weight decay for regularization
BOX_GAIN = 7.5                    # Weight for box loss
CLS_GAIN = 0.5                    # Weight for class prediction loss
DFL_GAIN = 1.5                    # Weight for directional feature loss
LABEL_SMOOTHING = 0.0             # Label smoothing value
DROPOUT = 0.0                     # Dropout rate
USE_VAL = True                    # Whether to use a validation set
SAVE_PLOTS = True                 # Save training progress plots
USE_PRETRAINED = True             # Use pretrained model weights

# data augmentations
HSV_H = 0.015       # Hue shift (in HSV color space)
HSV_S = 0.7         # Saturation shift
HSV_V = 0.4         # Value (brightness) shift
DEGREES = 0.0       # Max Rotation degrees
TRANSLATE = 0.1     # Translation
SCALE = 0.5         # Scaling
SHEAR = 0.0         # Shearing
PERSPECTIVE = 0.0   # Perspective transformation
FLIPUD = 0.0        # Vertical flip
FLIPLR = 0.5        # Horizontal flip
MOSAIC = 0.0        # Mosaic data augmentation
MIXUP = 0.0         # Mixup data augmentation

# print experiment name
print(EXP_NAME)

# experiment is logged on Wandb
training_results = model_exp1.train(
    # data
    data=local_ocr_ds_yaml_path,
    # hyperparameters
    imgsz=IMGSZ,
    epochs=EPOCHS,
    batch=BATCH,
    patience=PATIENCE,
    device=DEVICE,
    project=PROJECT_NAME,
    name=EXP_NAME,
    optimizer=OPTIMIZER,
    verbose=VERBOSE,
    fraction=FRACTION_TO_TRAIN,
    lr0=InitLR,
    lrf=FinalLR,
    momentum=MOMENTUM,
    weight_decay=WEIGHT_DECAY,
    box=BOX_GAIN,
    cls=CLS_GAIN,
    dfl=DFL_GAIN,
    label_smoothing=LABEL_SMOOTHING,
    dropout=DROPOUT,
    val=USE_VAL,
    plots=SAVE_PLOTS,
    pretrained=USE_PRETRAINED,
    seed=SEED,
    # data augmentations
    hsv_h=HSV_H,
    hsv_s=HSV_S,
    hsv_v=HSV_V,
    degrees=DEGREES,
    translate=TRANSLATE,
    scale=SCALE,
    shear=SHEAR,
    perspective=PERSPECTIVE,
    flipud=FLIPUD,
    fliplr=FLIPLR,
    mosaic=MOSAIC,
    mixup=MIXUP
)

baseline_yolov8n_ocr_20231206_112205
New https://pypi.org/project/ultralytics/8.0.222 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.0.221 🚀 Python-3.9.18 torch-2.1.1+cu118 CUDA:0 (NVIDIA GeForce RTX 3060, 12287MiB)
engine\trainer: task=detect, mode=train, model=yolov8n.pt, data=D:/College/Fall2023/DeepLearning/yolo_ocr/data.yaml, epochs=10, patience=50, batch=8, imgsz=640, save=True, save_period=-1, cache=False, device=0, workers=8, project=yolov8n_ocr, name=baseline_yolov8n_ocr_20231206_112205, exist_ok=False, pretrained=True, optimizer=AdamW, verbose=True, seed=42, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, ag

Freezing layer 'model.22.dfl.conv.weight'
AMP: running Automatic Mixed Precision (AMP) checks with YOLOv8n...
AMP: checks passed ✅


train: Scanning D:\College\Fall2023\DeepLearning\yolo_ocr\train\labels.cache... 4245 images, 0 backgrounds, 0 corrupt: 100%|██████████| 4245/4245 [00:00<?, ?it/s]

train: WARNING ⚠️ D:\College\Fall2023\DeepLearning\yolo_ocr\train\images\03c3dfc7e998b8562_jpg.rf.3d9d76c2437bcfb5cf433c0b6306d1ac.jpg: 1 duplicate labels removed
train: WARNING ⚠️ D:\College\Fall2023\DeepLearning\yolo_ocr\train\images\185d5dfa193c4ced_jpg.rf.b3d2ab34086b9e8f050b514cc22b4b9a.jpg: 1 duplicate labels removed
train: WARNING ⚠️ D:\College\Fall2023\DeepLearning\yolo_ocr\train\images\24c1fc0bf63da8153_jpg.rf.bd40f6e750b1a7990a9945aadddc1ef1.jpg: 1 duplicate labels removed
train: WARNING ⚠️ D:\College\Fall2023\DeepLearning\yolo_ocr\train\images\421ca0ce7f9329622_jpg.rf.4e25470bde67152fea6834e88e803708.jpg: 1 duplicate labels removed
train: WARNING ⚠️ D:\College\Fall2023\DeepLearning\yolo_ocr\train\images\4572990fd64bb6be3_jpg.rf.7ce23d5cd8b919a7832485c4815d97c8.jpg: 1 duplicate labels removed
train: WARNING ⚠️ D:\College\Fall2023\DeepLearning\yolo_ocr\train\images\492f6d607aae58c12_jpg.rf.44d8f6885cd8664eeff661809ad807ab.jpg: 1 duplicate labels removed
train: WARNING ⚠️ D:\Co


val: Scanning D:\College\Fall2023\DeepLearning\yolo_ocr\valid\labels.cache... 1221 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1221/1221 [00:00<?, ?it/s]

val: WARNING ⚠️ D:\College\Fall2023\DeepLearning\yolo_ocr\valid\images\0ecee0dd32848030_jpg.rf.ffe2fb185ba29646fcf16eacf0e99021.jpg: 1 duplicate labels removed
val: WARNING ⚠️ D:\College\Fall2023\DeepLearning\yolo_ocr\valid\images\22deb87614a920e6_jpg.rf.a72d572c1920415b880c030e7b1a17e6.jpg: 1 duplicate labels removed
val: WARNING ⚠️ D:\College\Fall2023\DeepLearning\yolo_ocr\valid\images\dcb210b4cae8edf22_jpg.rf.286248320e4fe0f26c75339865578ccc.jpg: 1 duplicate labels removed
val: WARNING ⚠️ D:\College\Fall2023\DeepLearning\yolo_ocr\valid\images\f7f068d4fb49a4f34_jpg.rf.8856a3518481e9dcc022b1a802a966cd.jpg: 1 duplicate labels removed


Plotting labels to yolov8n_ocr\baseline_yolov8n_ocr_20231206_112205\labels.jpg... 


Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x0000022D73C1BEE0>
Traceback (most recent call last):
  File "C:\Users\the_3\anaconda3\envs\yolo\lib\site-packages\torch\utils\data\dataloader.py", line 1478, in __del__
    self._shutdown_workers()
  File "C:\Users\the_3\anaconda3\envs\yolo\lib\site-packages\torch\utils\data\dataloader.py", line 1436, in _shutdown_workers
    if self._persistent_workers or self._workers_status[worker_id]:
AttributeError: '_MultiProcessingDataLoaderIter' object has no attribute '_workers_status'


optimizer: AdamW(lr=0.01, momentum=0.937) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to yolov8n_ocr\baseline_yolov8n_ocr_20231206_112205
Starting training for 10 epochs...
Closing dataloader mosaic


In [ ]:
# display the sum training results
training_results

### Evaluation (20 points)
- Evaluate your trained model's performance on a test dataset using mAP (mean Average Precision).
- Visualize the model's predictions on sample images.
- Analyze the model's performance, discussing any specific failure modes on your test dataset.

In [ ]:
# TODO: focus here
# evaluate the model on the test dataset
metrics = model.val(data=yaml_path, imgsz=640, plots=True, split='test')

In [ ]:
# display the metrics
metrics

In [ ]:
# visualize the model's predictions on sample images

##### Experiment 2
- Testing following hyperparameter updates

- To prevent overfitting, I use early stopping, weight decay, and dropout
- Image resizing (to IMGSZ) and normalization (to [0, 1]) is handle by ultralytics library

In [ ]:
# loading pre-trained yolov8 model from ultralytics
model_exp2 = YOLO('yolov8n.pt')
# display the model architecture
model_exp2

##### Experiment 3
- Testing following hyperparameter updates

- To prevent overfitting, I use early stopping, weight decay, and dropout
- Image resizing (to IMGSZ) and normalization (to [0, 1]) is handle by ultralytics library

In [ ]:
# loading pre-trained yolov8 model from ultralytics
model_exp3 = YOLO('yolov8n.pt')
# display the model architecture
model_exp3

##### Experiment 4
- Testing following hyperparameter updates

- To prevent overfitting, I use early stopping, weight decay, and dropout
- Image resizing (to IMGSZ) and normalization (to [0, 1]) is handle by ultralytics library

In [ ]:
# loading pre-trained yolov8 model from ultralytics
model_exp4 = YOLO('yolov8n.pt')
# display the model architecture
model_exp4

##### Final Run

In [ ]:
# loading pre-trained yolov8 model from ultralytics
final_model = YOLO('yolov8n.pt')
# display the model architecture
final_model

### BONUS (25 bonus points)
- Design and train a custom CNN-based object detection model using a self-designed architecture.
- Alternatively, compare several publicly available models on the same dataset.
- Consider implementing Gaussian YOLO V8 and comparing it against YOLO V8.

Please ensure that your code includes docstrings and type hints as requested, and be thorough in your analysis and experimentation.

### Setup and Imports

### First Training Experiment (baseline)

In [7]:
# Training.
results = model.train(
    data=yaml_path,
    imgsz=640,
    epochs=10000,
    batch=64,
    name='yolov8n_ocr')

New https://pypi.org/project/ultralytics/8.0.222 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.0.221 🚀 Python-3.9.18 torch-2.1.1+cu118 CUDA:0 (NVIDIA GeForce RTX 3060, 12287MiB)
engine\trainer: task=detect, mode=train, model=yolov8n.pt, data=D:/College/Fall2023/DeepLearning/yolo_ocr/data.yaml, epochs=10000, patience=50, batch=64, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=yolov8n_ocr3, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, show=False,

Freezing layer 'model.22.dfl.conv.weight'
AMP: running Automatic Mixed Precision (AMP) checks with YOLOv8n...
AMP: checks passed ✅


train: Scanning D:\College\Fall2023\DeepLearning\yolo_ocr\train\labels.cache... 4245 images, 0 backgrounds, 0 corrupt: 100%|██████████| 4245/4245 [00:00<?, ?it/s]

train: WARNING ⚠️ D:\College\Fall2023\DeepLearning\yolo_ocr\train\images\03c3dfc7e998b8562_jpg.rf.3d9d76c2437bcfb5cf433c0b6306d1ac.jpg: 1 duplicate labels removed
train: WARNING ⚠️ D:\College\Fall2023\DeepLearning\yolo_ocr\train\images\185d5dfa193c4ced_jpg.rf.b3d2ab34086b9e8f050b514cc22b4b9a.jpg: 1 duplicate labels removed
train: WARNING ⚠️ D:\College\Fall2023\DeepLearning\yolo_ocr\train\images\24c1fc0bf63da8153_jpg.rf.bd40f6e750b1a7990a9945aadddc1ef1.jpg: 1 duplicate labels removed
train: WARNING ⚠️ D:\College\Fall2023\DeepLearning\yolo_ocr\train\images\421ca0ce7f9329622_jpg.rf.4e25470bde67152fea6834e88e803708.jpg: 1 duplicate labels removed
train: WARNING ⚠️ D:\College\Fall2023\DeepLearning\yolo_ocr\train\images\4572990fd64bb6be3_jpg.rf.7ce23d5cd8b919a7832485c4815d97c8.jpg: 1 duplicate labels removed
train: WARNING ⚠️ D:\College\Fall2023\DeepLearning\yolo_ocr\train\images\492f6d607aae58c12_jpg.rf.44d8f6885cd8664eeff661809ad807ab.jpg: 1 duplicate labels removed
train: WARNING ⚠️ D:\Co


val: Scanning D:\College\Fall2023\DeepLearning\yolo_ocr\valid\labels.cache... 1221 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1221/1221 [00:00<?, ?it/s]

...

                   all       1221       7890      0.961      0.941      0.975      0.836
Stopping training early as no improvement observed in last 50 epochs. Best results observed at epoch 143, best model saved as best.pt.
To update EarlyStopping(patience=50) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.



193 epochs completed in 2.306 hours.
Optimizer stripped from runs\detect\yolov8n_ocr3\weights\last.pt, 6.3MB
Optimizer stripped from runs\detect\yolov8n_ocr3\weights\best.pt, 6.3MB

Validating runs\detect\yolov8n_ocr3\weights\best.pt...
Ultralytics YOLOv8.0.221 🚀 Python-3.9.18 torch-2.1.1+cu118 CUDA:0 (NVIDIA GeForce RTX 3060, 12287MiB)
Model summary (fused): 168 layers, 3012668 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 10/10 [00:12<00:00,  1.26s/it]


                   all       1221       7890      0.967      0.941      0.977      0.837
                     0       1221        408      0.863      0.931      0.933      0.786
                     1       1221        386      0.923      0.927      0.963      0.736
                     2       1221        399      0.985      0.975      0.992      0.843
                     3       1221        369      0.997      0.989      0.993      0.843
                     4       1221        326      0.988      0.951      0.987      0.811
                     5       1221        383      0.984      0.976      0.989      0.838
                     6       1221        393      0.991      0.977      0.991      0.848
                     7       1221        343       0.99       0.98      0.992      0.826
                     8       1221        346      0.973       0.96      0.986      0.834
                     9       1221        324      0.988      0.978      0.993      0.857
                     

lr/pg0,▁███████████████████████████████████████
lr/pg1,▁███████████████████████████████████████
lr/pg2,▁███████████████████████████████████████
metrics/mAP50(B),▂▁▃▆▆▇▇▇▇▇█▇████████████████████████████
metrics/mAP50-95(B),▂▁▂▄▅▅▆▆▆▆▇▇▇▇▇▇▇▇▇█████████████████████
metrics/precision(B),▃▁▁▄▅▅▇▅▆▆▆▆▇▇▇▇▇▇▇▆▇▇▇▇▇▇█▇█▇▇▇██▇██▇▇▇
metrics/recall(B),▃▁▃▅▆▆▇▇▇▇▇▇▇▇█▇▇▇██████████████████▇███
model/GFLOPs,▁
model/parameters,▁
model/speed_PyTorch(ms),▁
train/box_loss,██▇▆▅▅▄▄▄▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▁▂▁▁▁▁▁▁▁▁▁▁▁▁


### Evaluation

In [12]:
# evaluate the model on the test dataset
metrics = model.val(data=yaml_path, imgsz=640, plots=True, split='test')

Ultralytics YOLOv8.0.221 🚀 Python-3.9.18 torch-2.1.1+cu118 CUDA:0 (NVIDIA GeForce RTX 3060, 12287MiB)


val: Scanning D:\College\Fall2023\DeepLearning\yolo_ocr\test\labels.cache... 610 images, 0 backgrounds, 0 corrupt: 100%|██████████| 610/610 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 10/10 [00:46<00:00,  4.62s/it]


                   all        610       3954      0.951      0.926      0.961      0.756
                     0        610        231       0.84      0.931      0.898      0.705
                     1        610        239      0.913      0.874      0.937      0.672
                     2        610        191      0.987       0.99      0.994      0.782
                     3        610        186      0.991      0.968      0.988      0.788
                     4        610        160      0.961      0.981      0.991      0.785
                     5        610        205       0.99      0.985      0.995      0.804
                     6        610        226      0.995      0.978      0.989      0.789
                     7        610        197      0.985      0.977      0.987      0.755
                     8        610        177      0.962      0.972      0.992      0.786
                     9        610        185      0.984      0.988      0.994      0.806
                     

### Custom Yolo G8 Architecture in Pytorch (NO UltraLytics here unfortunately)